# Imports

In [66]:
import cv2
import imageio.v3 as iio
import cv2 
import matplotlib.pyplot as plt
import numpy as np
import time

# 1. Video Acquisition  

# 2. Frame Extraction

In [67]:
def get_frame_indices(video_path:str, num_frames:int) -> list:
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    if num_frames > total_frames:
        raise ValueError(f"Requested {num_frames} frames, but video has only {total_frames} frames.")

    # Compute evenly spaced indices
    indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    return indices.tolist()

def load_specific_frames(video_path:str, selected_indices:list, display_frames=True) -> list:
    frames = []
    selected_set = set(selected_indices)
    frame_count = 0

    try:
        for frame in iio.imiter(video_path):
            if frame_count in selected_set:
                frames.append(frame)

                if display_frames:
                    plt.imshow(frame)
                    plt.title(f'Frame {frame_count}')
                    plt.axis('off')
                    plt.show()

            frame_count += 1

    except FileNotFoundError:
        print(f"Error: Video file not found at {video_path}")
        return []
    except Exception as e:
        print(f"An error occurred while processing the video: {e}")
        return []

    print(f"\nFinished loading selected frames.")
    print(f"Total frames processed: {frame_count}")
    
    return frames

In [ ]:
video_path = "../videos/vid1.mp4"
indices = get_frame_indices(video_path, num_frames=15)
frames = load_specific_frames(video_path, selected_indices=indices, display_frames=False)
pair_indices = list(range(len(frames) - 1))  # Pairs: (0,1), (1,2), ..., (n-2, n-1)


Finished loading selected frames.
Total frames processed: 291


# 3. Feature Detection and Description

## Models

In [69]:
def detect_orb(gray):
    orb = cv2.ORB_create(nfeatures=3000)
    return orb.detect(gray, None)

def detect_sift(gray):
    sift = cv2.SIFT_create(nfeatures=3000, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
    keypoints, _ = sift.detectAndCompute(gray, None)
    return keypoints

def detect_fast(gray):
    fast = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
    return fast.detect(gray, None)

def detect_shi_tomasi(gray):
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.01, minDistance=10, blockSize=3)
    if corners is not None:
        return [cv2.KeyPoint(float(x), float(y), 1) for [[x, y]] in corners]
    return []

def detect_harris(gray, block_size=2, ksize=3, k=0.04, threshold_ratio=0.01):
    gray_f32 = np.float32(gray)
    dst = cv2.cornerHarris(gray_f32, block_size, ksize, k)
    dst = cv2.dilate(dst, None)
    threshold = threshold_ratio * dst.max()
    corners = np.argwhere(dst > threshold)
    return [cv2.KeyPoint(float(pt[1]), float(pt[0]), 1) for pt in corners]


## Visualization

In [ ]:
def run_feature_detector_pairs(frames, pair_indices, detector_fn, title_prefix="Detector"):
    #pair_indices = list(pair_indices)  # Ensure it's a list, not a numpy array

    if len(frames) < 2:
        print("Error: At least two frames are necessary.")
        return

    max_index = len(frames) - 2
    invalid_indices = [i for i in pair_indices if i < 0 or i > max_index]
    if invalid_indices:
        print(f"Error: Invalid indices found: {invalid_indices}. Must be in range 0 to {max_index}.")
        return

    for idx, i in enumerate(pair_indices, 1):
        j = i + 1
        img1 = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(frames[j], cv2.COLOR_BGR2GRAY)

        keypoints1 = detector_fn(img1)
        keypoints2 = detector_fn(img2)

        print(f"\nPair {idx}: Frame {i} and Frame {j}")
        print(f"- Keypoints in Frame {i}: {len(keypoints1)}")
        print(f"- Keypoints in Frame {j}: {len(keypoints2)}")

        img_kp1 = cv2.drawKeypoints(img1, keypoints1, None, color=(0, 255, 0))
        img_kp2 = cv2.drawKeypoints(img2, keypoints2, None, color=(0, 255, 0))

        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        axes[0].imshow(img_kp1, cmap='gray')
        axes[0].set_title(f"{title_prefix} - Frame {i}")
        axes[1].imshow(img_kp2, cmap='gray')
        axes[1].set_title(f"{title_prefix} - Frame {j}")

        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])

        plt.suptitle(f"{title_prefix} Comparison: Pair {idx}")
        plt.tight_layout()
        plt.show()

### ORB

In [ ]:
run_feature_detector_pairs(frames, pair_indices, detect_orb, title_prefix="ORB")

Error: Invalid indices found: [20, 41, 61, 82, 102, 123, 144, 164, 185, 205, 226, 246, 267, 288]. Must be in range 0 to 13.


### SIFT

In [ ]:
run_feature_detector_pairs(frames, pair_indices, detect_sift, title_prefix="SIFT")

Error: Invalid indices found: [20, 41, 61, 82, 102, 123, 144, 164, 185, 205, 226, 246, 267, 288]. Must be in range 0 to 13.


### FAST

In [ ]:
run_feature_detector_pairs(frames, pair_indices, detect_fast, title_prefix="FAST")

Error: Invalid indices found: [20, 41, 61, 82, 102, 123, 144, 164, 185, 205, 226, 246, 267, 288]. Must be in range 0 to 13.


### SHI-TOMASI

In [ ]:
run_feature_detector_pairs(frames, pair_indices, detect_shi_tomasi, title_prefix="Shi-Tomasi")

Error: Invalid indices found: [20, 41, 61, 82, 102, 123, 144, 164, 185, 205, 226, 246, 267, 288]. Must be in range 0 to 13.


### Harris

In [ ]:
run_feature_detector_pairs(frames, pair_indices, detect_harris, title_prefix="Harris")

Error: Invalid indices found: [20, 41, 61, 82, 102, 123, 144, 164, 185, 205, 226, 246, 267, 288]. Must be in range 0 to 13.


# 4. Feature Matching and Outlier Rejection 

# 5. Essential/Fundamental Matrix Computation 

# 6. Camera Pose Estimation

# 7. 3D Point Triangulation and Scene Visualisation 

# 8.Evaluation and Analysis 